In [141]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import folium 
import pandas as pd
import folium.plugins as plugins

In [142]:
BD = pd.read_csv('dados-curso-completo.csv')

In [143]:
BD.head()

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
0,2020-03-23 18:51:14,Gaboardi,fraslee,@BolsonaroSP Cade o fundão deputado??? Congres...,0,138,pt,NaN,NaN,NaN,NaN,NaN,[]
1,2020-03-18 03:18:14,Seu Zé da Farmácia,seuzedafarmacia,"Em ação preventiva, o Governo do Estado suspen...",0,56,pt,NaN,NaN,NaN,NaN,NaN,[]
2,2020-03-29 12:09:27,Marília Oliveira 🇧🇷,Marioliveira_gb,Incrível como os defensores do fim do distanci...,0,5,pt,NaN,NaN,NaN,NaN,NaN,[]
3,2020-04-01 21:11:07,🚩,santosodeto,Fumaça de cigarro pode propagar coronavírus? h...,0,1885,pt,NaN,NaN,NaN,NaN,NaN,[]
4,2020-03-19 17:49:40,Nova Impressão,novaimpressao_,🚨Todos juntos contra COVID-19💪\n\nSolicite seu...,0,124,pt,Brasília,Brazil,BR,-47.8778,-15.77691,[]


In [101]:
BD

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
0,2020-03-23 18:51:14,Gaboardi,fraslee,@BolsonaroSP Cade o fundão deputado??? Congres...,0,138,pt,NaN,NaN,NaN,NaN,NaN,[]
1,2020-03-18 03:18:14,Seu Zé da Farmácia,seuzedafarmacia,"Em ação preventiva, o Governo do Estado suspen...",0,56,pt,NaN,NaN,NaN,NaN,NaN,[]
2,2020-03-29 12:09:27,Marília Oliveira 🇧🇷,Marioliveira_gb,Incrível como os defensores do fim do distanci...,0,5,pt,NaN,NaN,NaN,NaN,NaN,[]
3,2020-04-01 21:11:07,🚩,santosodeto,Fumaça de cigarro pode propagar coronavírus? h...,0,1885,pt,NaN,NaN,NaN,NaN,NaN,[]
4,2020-03-19 17:49:40,Nova Impressão,novaimpressao_,🚨Todos juntos contra COVID-19💪\n\nSolicite seu...,0,124,pt,Brasília,Brazil,BR,-47.8778,-15.77691,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658820,2020-03-13 17:26:18,broca,whoisdrill,oms: tamo tendo pandemia evitar\n conglomerado...,0,361,pt,NaN,NaN,NaN,NaN,NaN,[]
1658821,2020-05-12 13:02:34,Renato 🇧🇷,VerdadeExpress,"@OsmarTerra Sr. Osmar Terra, gostaria de ouvir...",0,4066,pt,NaN,NaN,NaN,NaN,NaN,[]
1658822,2020-03-19 18:16:30,eugeniocosta8,EugenioCosta8,CIENTISTA DE HARVARD PREVÊ QUE ATÉ 70% DO MUND...,3,5307,pt,NaN,NaN,NaN,NaN,NaN,['CORONAVIRUS']
1658823,2020-04-07 18:14:45,Wagner Mota - Seriedade + Geek,wagnermota75,"#BolsonaroTemRazaoSim porque, quando a prefeit...",0,1000,pt,NaN,NaN,NaN,NaN,NaN,['BolsonaroTemRazaoSim']


In [144]:
BD.duplicated().sum() #verificando linhas iguais

2

In [145]:
BD.isnull().sum() #contando (identificando) dados faltantes

data                0
usuario            57
apelido             0
texto               0
retweet             0
seguidores          0
idioma              0
lugar         1584220
pais          1584225
sigla         1584230
latitude      1650105
longitude     1650105
hashtags            0
dtype: int64

In [146]:
BD.dropna (inplace=True) #limpando dados nulos de todo o banco
BD.isnull().sum()

data          0
usuario       0
apelido       0
texto         0
retweet       0
seguidores    0
idioma        0
lugar         0
pais          0
sigla         0
latitude      0
longitude     0
hashtags      0
dtype: int64

In [147]:
banco=BD.loc[BD['idioma']=='pt'] #do banco completo, pegando só o que está em PT
banco

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
4,2020-03-19 17:49:40,Nova Impressão,novaimpressao_,🚨Todos juntos contra COVID-19💪\n\nSolicite seu...,0,124,pt,Brasília,Brazil,BR,-47.877800,-15.776910,[]
261,2020-04-16 00:34:54,Felipe Silva,felipesilvasr,A secretaria de Saúde da Paraíba confirmou nes...,0,150,pt,Taperoá,Brazil,BR,-36.825000,-7.162778,[]
265,2020-05-03 13:48:52,Bruno Fortes,Brunaogf,Domingão taí! ☀️\nEm casa! 😐😬\nVai passar! 🙏🏼\...,0,120,pt,Fortaleza,Brazil,BR,-38.526600,-3.729300,"['emcasa', 'isolamentosocial', 'saude', 'quero..."
289,2020-03-31 17:52:37,jairo santos souza,soi_jairo,"Seguindo orientação do Presidente Bolsonaro, o...",0,22,pt,Euclides da Cunha,Brazil,BR,-39.014620,-10.505250,[]
477,2020-03-24 17:27:29,Carolina Porne,carolinaporne,#5 - Isolamento social não é desculpa para fic...,0,373,pt,Guarulhos,Brazil,BR,-46.495600,-23.445600,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658284,2020-04-16 19:45:29,Bruno Agrofoglio Ferreira,agrofoglio,Criei um bot para o Telegram raspando os dados...,0,49,pt,Paulínia,Brazil,BR,-47.203010,-22.757163,[]
1658503,2020-05-31 12:38:32,BARÃO DE CEARA MIRIM,baraocearamirim,Serra da Formiga em Riachuelo RN\nDestino marc...,0,808,pt,Riachuelo,Brazil,BR,-35.823770,-5.815792,[]
1658514,2020-03-23 23:48:19,Diego Pelizari,diegopelizari,#Repost de respeito da parceira agroschoolbras...,0,174,pt,São Félix do Araguaia,Brazil,BR,-53.000000,-11.000000,['Repost']
1658760,2020-05-12 09:30:34,Hamilton Valerio,hvalerio,Mais um amanhecer em isolamento social. em Pra...,0,176,pt,Jaboatão dos Guararapes,Brazil,BR,-34.919555,-8.198139,[]


In [168]:
banco.filter(items=['apelido','pais']).groupby('pais').count().sort_values(by='pais',ascending=False) #tentando filtrar e ordenar por quantidade

,apelido
pais,
Vietnam,3
Venezuela,1
Vatican City,3
Uruguay,1
United States,94
United Kingdom,12
Turkey,1
The Netherlands,1
Thailand,2


In [176]:
banco[banco['texto'].str.contains('covid')]

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
2469,2020-06-26 18:02:23,Augusto Delfim Moreira,Augustodelfimmo,Continuamos na Fase Laranja #sjcampos #sjc #co...,0,20,pt,São José dos Campos,Brazil,BR,-45.886944,-23.178889,"['sjcampos', 'sjc', 'covid19']"
3420,2020-04-09 22:32:22,Alexandre Cezar Fh,AlexandreCezarf,Mais que #irmaos ...\nSomos #siblings !!!\n\na...,0,15,pt,Curitiba,Brazil,BR,-49.276342,-25.428955,"['irmaos', 'siblings', 'quarentinewithme', 'qu..."
8310,2020-03-13 21:02:00,STEⓋE BOZIC,STEVE_BOZIC,#CANADA #covid_19 #covıd19 #coronavirus #truth...,0,466,pt,Toronto,Canada,CA,-79.387200,43.648000,"['CANADA', 'covid_19', 'covıd19', 'coronavirus..."
12194,2020-03-29 18:52:50,x Fabim x ︻┳ テ = 一,xfabimx,Né Alice uma piscina e um sorvete nem cai bem ...,0,590,pt,Sao Paulo,Brazil,BR,-46.649423,-23.594328,['covid19']
14110,2020-05-14 14:55:00,BLACK BEAUTIFUL,Reinald43867320,Hoje a nossa história matutino em combatê ao c...,0,15,pt,Ilhéus,Brazil,BR,-39.062848,-14.752700,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648773,2020-02-29 14:03:33,Mark Hillary (em 🏠),markhillary,São Paulo coronavírus #máscara 😷 #saopaulo #co...,0,17645,pt,Sao Paulo,Brazil,BR,-46.665490,-23.564160,"['máscara', 'saopaulo', 'covid19', 'coronavirus']"
1653097,2020-03-23 14:23:04,Vamu Ver!,Vamu_Ver,"Enquanto isso, após duas semanas em isolamento...",0,639,pt,Lisbon,Portugal,PT,-9.183333,38.700000,"['vamuver', 'isolamentosocial', 'covid19portug..."
1654902,2020-05-24 12:07:02,Cristiano Ricelli,cristianoriceli,"O ex-prefeito de Patos e ex-deputado, Dinaldo ...",1,276,pt,João Pessoa,Brazil,BR,-34.862700,-7.120700,[]
1654971,2020-04-17 01:57:31,Zap Bolsonaro,BolsonaroZap,Chegaram as máscaras para os petistas. #covid1...,0,131,pt,Boston,United States,US,-71.056500,42.357700,['covid19']


In [202]:
pergunta1=banco[banco['texto'].str.contains('governo')].count() #contando quantas vezes a palavra 'presidente' é cidada.
pergunta2=banco[banco['texto'].str.contains('Governo')].count()
p=pergunta1+pergunta2
p


data          145
usuario       145
apelido       145
texto         145
retweet       145
seguidores    145
idioma        145
lugar         145
pais          145
sigla         145
latitude      145
longitude     145
hashtags      145
dtype: int64

In [204]:
prop1=(p/len(banco))*100 #proporção da palavra no conjunto
prop1

data          1.691357
usuario       1.691357
apelido       1.691357
texto         1.691357
retweet       1.691357
seguidores    1.691357
idioma        1.691357
lugar         1.691357
pais          1.691357
sigla         1.691357
latitude      1.691357
longitude     1.691357
hashtags      1.691357
dtype: float64

In [205]:
ministro1=banco[banco['texto'].str.contains('ministro')].count() #contando quantas vezes a palavra 'presidente' é cidada.
ministro2=banco[banco['texto'].str.contains('Ministro')].count()
M=ministro1+ministro2
M

data          39
usuario       39
apelido       39
texto         39
retweet       39
seguidores    39
idioma        39
lugar         39
pais          39
sigla         39
latitude      39
longitude     39
hashtags      39
dtype: int64

In [210]:
prop2=(M/len(banco))*100
prop2[1]

0.4549165986235857

In [207]:
ministro1=banco[banco['texto'].str.contains('ministro')].count() #contando quantas vezes a palavra 'presidente' é cidada.
ministro2=banco[banco['texto'].str.contains('Ministro')].count()
M=ministro1+ministro2
M

data          39
usuario       39
apelido       39
texto         39
retweet       39
seguidores    39
idioma        39
lugar         39
pais          39
sigla         39
latitude      39
longitude     39
hashtags      39
dtype: int64

In [213]:
d = np.array([prop1[1],prop2[1]])
d

array([1.69135658, 0.4549166 ])

In [82]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [16]:
mapa=folium.Map(location=[-15.776910,-47.87780],
  tiles='Stamen Terrain', zoom_start=10)
mapa

In [17]:
folium.Marker([-15.776910,-47.87780],tooltip='Clique aqui!', icon=folium.Icon(color='green')).add_to(mapa)#marcador comum

In [18]:
folium.CircleMarker(location=[-15.776910,-47.87780],
                   radius=10,
                   color='#3186cc',
                   fill=True,
                   fill_color='#3186cc').add_to(mapa)

In [19]:
coor=[]
for lat,lng in zip (BD.longitude.values[:1000],banco.latitude.values[:1000]):coor.append([lat,lng])

In [23]:
mapa=folium.Map(location=[-15.788497,-47.879873],zoom_start=10,titles='Stamen Torner')

In [24]:
mapa.add_child(plugins.HeatMap(coor))
mapa

In [22]:
len(coor)

518

In [57]:
palavras = ['teste', 'assistente', 'estou', 'aqui', 'onde', 'ouro', 'adeus']
def f(p):
    return p.startswith('a') or p.startswith('e')

    print filter(f, palavras)


SyntaxError: invalid syntax (<ipython-input-57-dd68891afe07>, line 5)

In [63]:
open('newfile','w').writelines([ line for line in open('textfile') if 'Apple' in line])



TypeError: expected str, bytes or os.PathLike object, not Series

In [61]:
def f(p):
    return p.startswith('a') or p.startswith('e')

    print filter(f, palavras)


SyntaxError: invalid syntax (<ipython-input-61-866e0d0dc277>, line 4)

In [59]:
 print filter(f, palavras)

SyntaxError: invalid syntax (<ipython-input-59-593e95622846>, line 1)